In [7]:
# Dependencies
import pandas as pd
import requests
import json
import os
import datetime
import matplotlib.pyplot as plt
import numpy as np

# Google API Key (maybe later)
from config import gkey


In [8]:
# Source files
attendance_raw = 'Resources/national_park_attendance_Prev3Y.xlsx'
park_info = 'Resources/national_park_locations.xlsx'
#wildfires = 'Resources/California_Fire_Incidents.csv'

# Read data and store in dataframe
attendance_data = pd.read_excel(attendance_raw)
park_info_data = pd.read_excel(park_info)
#wildfire_data = pd.read_csv(wildfires)


In [9]:
# Limit dataframe to California & Florifa
national_parks = park_info_data.loc[(park_info_data['ParkType']=='National Park')]
focus_parks = pd.DataFrame(national_parks.loc[(park_info_data['State'] == 'CA') | (park_info_data['State'] == 'OR') | (park_info_data['State'] == 'WA')])
focus_parks.reset_index()

#focus_parks['FullName'] = focus_parks['Park']

# Create new 'FullName' column (empty)
focus_parks['FullName'] = focus_parks['Park'].str.replace(['NP'],'')#+focus_parks['ParkType'])

focus_parks['FullName']=focus_parks['Park'].str[:-2]+focus_parks['ParkType']

#focus_parks['Park'].replace('NP', 'National Park', inplace=True)
#focus_parks['Park'] = focus_parks['Park'].str.replace('NP','')

focus_parks.head(20)

,Park,UnitCode,ParkType,Region,State,FullName
63,Channel Islands NP,CHIS,National Park,Pacific West,CA,Channel Islands National Park
79,Crater Lake NP,CRLA,National Park,Pacific West,OR,Crater Lake National Park
87,Death Valley NP,DEVA,National Park,Pacific West,CA,Death Valley National Park
191,Joshua Tree NP,JOTR,National Park,Pacific West,CA,Joshua Tree National Park
198,Kings Canyon NP,KICA,National Park,Pacific West,CA,Kings Canyon National Park
210,Lassen Volcanic NP,LAVO,National Park,Pacific West,CA,Lassen Volcanic National Park
244,Mount Rainier NP,MORA,National Park,Pacific West,WA,Mount Rainier National Park
262,North Cascades NP,NOCA,National Park,Pacific West,WA,North Cascades National Park
265,Olympic NP,OLYM,National Park,Pacific West,WA,Olympic National Park
281,Pinnacles NP,PINN,National Park,Pacific West,CA,Pinnacles National Park


In [ ]:
# Find the geocoordinates (latitude and longitude)
park = "Yellowstone"

params = {"address": park, "key": gkey}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

#print("Drill #1: The Geocoordinates of X")

# Run request
response = requests.get(base_url, params=params)

# print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
# print(response.url)

# Convert to JSON
park_geo = response.json()

# Extract lat/lng
lat = park_geo["results"][0]["geometry"]["location"]["lat"]
lng = park_geo["results"][0]["geometry"]["location"]["lng"]

# Print results
print(f"{park}: {lat}, {lng}")

In [ ]:
# Build URL using the Google Maps API

lats=[]
longs=[]
cost=[]

base_url = "https://maps.googleapis.com/maps/api/geocode/json"

for Park in focus_parks['FullName']:
    params = {"address": Park, "key": gkey}
    response = requests.get(base_url, params=params)
    park_geo = response.json()
    lats.append(park_geo["results"][0]["geometry"]["location"]["lat"])
    longs.append(park_geo["results"][0]["geometry"]["location"]["lng"])           
    

In [ ]:
# Add geocoordinates to dataframe
focus_parks['Latitude']=lats
focus_parks['Longitude']=longs
focus_parks.head()

In [ ]:
# Combine the data into a single dataframe  
park_data = pd.merge(focus_parks, attendance_data, how="left", on=["Park", "Park"])
park_data.head()

In [ ]:
park_data['Calendar Year'] = pd.DatetimeIndex(park_data['Month']).year
park_data['Calendar Month'] = pd.DatetimeIndex(park_data['Month']).month

park_data = park_data.set_index('FullName')

park_data.head(50)

In [ ]:
# Generate a bar plot - 
argh_group = park_data.groupby('Month')
measures = pd.DataFrame(argh_group['Visitors'].sum())

drug_chart=measures.plot(kind='bar', title='National Park Visitors', color='navy')

drug_chart.set_xlabel('Month')
drug_chart.set_ylabel('Total Visitors')

plt.show()

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the visutors for each calendar year
# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 


mean = park_data.groupby('Calendar Year').mean()['Visitors']
median = park_data.groupby('Calendar Year').median()['Visitors']
variance = park_data.groupby('Calendar Year').var()['Visitors']
std_dev = park_data.groupby('Calendar Year').std()['Visitors']
SEM = park_data.groupby('Calendar Year').sem()['Visitors']

# Assemble the resulting series into a single summary dataframe.
park_stats = pd.DataFrame({'Mean': mean,
              'Median':median,
              'Variance': variance,
              'Standard Deviation': std_dev,
                         'SEM':SEM})
park_stats.head(12)

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
# Using the aggregation method, produce the same summary statistics in a single line
# Google take the wheel ...
summary_stats= park_data.groupby('Park').agg({'Visitors':['mean','median','var','std','sem']})
summary_stats

In [ ]:
# Split up our data into groups based upon 'Calendar Year' and 'Calendar Month'
year_groups = park_data.groupby(['Calendar Year','Calendar Month'])

# Create a new variable that holds the sum of our groups
monthly_summary = year_groups.sum()
monthly_summary.head(50)
